In [1]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [2]:
df_train = pd.read_csv('train_with_features.csv')

## TODO: fill with 0 or median depending on the column

In [3]:
df_train = df_train.fillna(0)

In [4]:
kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)


In [5]:
df_train.stock_id = df_train.stock_id.astype('category')

In [6]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [7]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [16]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200],config={'lin_first':False},
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        #cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20),
                        #    SaveModelCallback(monitor='rmspe', comp=np.less, fname=fname)]
                           )
    learn.fit_one_cycle(70, 5e-3)
    learn.save(fname)
    return 0#learn.early_stopping.best

In [17]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [18]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [19]:
res =[]
for trn_idx, val_idx in kfold.split(df_train):
    res.append(train_fold(trn_idx, val_idx, f'fold2_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.256935,0.253195,0.256056,00:04
1,0.242970,0.239022,0.242633,00:03
2,0.243659,0.227246,0.229596,00:03
3,0.241277,0.243375,0.244691,00:04
4,0.247703,0.236285,0.237992,00:04
5,0.246728,0.236869,0.240856,00:04
6,0.240647,0.222112,0.224443,00:04
7,0.239649,0.226186,0.228747,00:04
8,0.233069,0.223464,0.226065,00:03
9,0.240022,0.223092,0.225443,00:04


fold 1 0


epoch,train_loss,valid_loss,rmspe,time
0,0.249557,0.231695,0.233534,00:03
1,0.246039,0.229118,0.231198,00:03
2,0.246101,0.226944,0.230048,00:04
3,0.239336,0.239686,0.241547,00:03
4,0.238995,0.225751,0.230059,00:03
5,0.236003,0.228373,0.231019,00:03
6,0.238821,0.224613,0.226173,00:03
7,0.241841,0.224223,0.226190,00:03
8,0.248753,0.245879,0.248017,00:04
9,0.238556,0.222071,0.224137,00:04


fold 2 0


epoch,train_loss,valid_loss,rmspe,time
0,0.265369,0.241658,0.246128,00:04
1,0.245770,0.228685,0.231258,00:03
2,0.244854,0.229438,0.231908,00:03
3,0.252749,0.249110,0.251725,00:04
4,0.236655,0.236016,0.238832,00:03
5,0.235060,0.232059,0.234807,00:03
6,0.238466,0.230368,0.232458,00:03
7,0.241269,0.234987,0.237285,00:03
8,0.243397,0.232689,0.235106,00:04
9,0.234171,0.226348,0.228154,00:04


...

epoch,train_loss,valid_loss,rmspe,time
0,0.253452,0.234282,0.236516,00:04
1,0.244739,0.229162,0.231667,00:04
2,0.238113,0.225006,0.228267,00:04
3,0.242084,0.227346,0.230198,00:04
4,0.237861,0.232135,0.234483,00:04
5,0.246257,0.266154,0.269047,00:03
6,0.245228,0.236683,0.257917,00:04
7,0.241464,0.224565,0.227105,00:04
8,0.242768,0.231772,0.234994,00:04
9,0.238122,0.253749,0.258493,00:03


fold 4 0


epoch,train_loss,valid_loss,rmspe,time
0,0.256374,0.233744,0.240966,00:04
1,0.249122,0.233625,0.240044,00:04
2,0.244962,0.236058,0.238003,00:04
3,0.235653,0.242498,0.245847,00:04
4,0.245423,0.243371,0.245960,00:04
5,0.243557,0.230872,0.234167,00:04
6,0.245481,0.230857,0.233314,00:04
7,0.241932,0.244352,0.247548,00:04
8,0.242931,0.227738,0.230708,00:04
9,0.236693,0.226425,0.228817,00:04


fold 5 0


(0.0, [0, 0, 0, 0, 0])